In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/Shared\ drives/ecl-pedro/video-coding-utils/notebooks/

---

In [1]:
from utils.utils import *
from utils import models

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.image import psnr
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [6]:
def PSNR(x,y):
    return psnr(x,y,max_val=1)

In [7]:
ds = tf.data.Dataset.from_tensor_slices(list(range(50000)))

In [8]:
ds = ds.shuffle(50000)

In [9]:
def parse_function(xdir, ydir):
    
    def fun(i):
        
        i = tf.strings.as_string(i)
        fcompressed = tf.io.read_file(xdir + i + '.png')
        compressed = tf.image.decode_png(fcompressed, channels=1)
        compressed = tf.image.convert_image_dtype(compressed, tf.float32)
        compressed = tf.image.resize(compressed, [64, 64])
        
        fgroundtruth = tf.io.read_file(ydir + i + '.png')
        groundtruth = tf.image.decode_png(fgroundtruth, channels=1)
        groundtruth = tf.image.convert_image_dtype(groundtruth, tf.float32)
        groundtruth = tf.image.resize(groundtruth, [64, 64])
        
        return compressed, groundtruth
    
    return fun

In [10]:
ds = ds.map(parse_function('../../data/preprocessed/basketball/compressed/', '../../data/preprocessed/basketball/ground_truth/'))

In [11]:
bds = ds.batch(128)

In [12]:
model = models.QECNN_I()

In [13]:
model.build((None, 64, 64, 1))

In [14]:
model.compile(
    optimizer='SGD',
    loss=MSE,
    metrics=[PSNR]
)

In [16]:
ckpt_callback = ModelCheckpoint(
    filepath='checkpoints/QECNN_I',
    save_weights_only=True,
    verbose=0
)

In [17]:
history = model.fit(
    bds,
    epochs=1,
    callbacks=ckpt_callback
)

ValueError: in user code:

    /home/pedro/miniconda3/envs/ecl/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /home/pedro/miniconda3/envs/ecl/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/pedro/miniconda3/envs/ecl/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/pedro/miniconda3/envs/ecl/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/pedro/miniconda3/envs/ecl/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/pedro/minicon